In [1]:
import numpy as np
import pandas as pd

from random import randint
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

from skimage.transform import resize

from keras.preprocessing.image import load_img
from keras import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input,Dense, AveragePooling2D,Conv2D, Conv2DTranspose,Flatten, MaxPooling2D, concatenate, Dropout,BatchNormalization,LeakyReLU,Activation,Add
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
from tqdm import tqdm_notebook
from keras import backend as K
from keras import optimizers
import time

Using TensorFlow backend.


In [2]:
data_df = pd.read_csv('./easy_more2.csv',index_col=0)

In [3]:
data = data_df.sample(frac=1).reset_index(drop=True)

In [4]:
img_path = '../cropped_GT'

In [5]:
imgs = os.listdir(img_path)

In [6]:
img_dic = {}
for img in imgs:
    img_dic[img] = np.array(load_img(img_path+'/'+img))/255
    print(img)
print('Read_Done')

140_d.jpg
016_dt.jpg
102_dt.jpg
054_t.jpg
171_d.jpg
164_.jpg
091_.jpg
048_t.jpg
020_dt.jpg
180_dt.jpg
177_dt.jpg
187_t.jpg
001_.jpg
064_dt.jpg
190_dt.jpg
118_dt.jpg
186_dt.jpg
064_.jpg
188_t.jpg
090_t.jpg
098_.jpg
179_t.jpg
033_d.jpg
052_d.jpg
048_d.jpg
160_d.jpg
066_dt.jpg
096_d.jpg
088_.jpg
152_d.jpg
136_t.jpg
143_.jpg
119_d.jpg
013_.jpg
006_.jpg
192_d.jpg
062_t.jpg
179_dt.jpg
193_t.jpg
172_.jpg
095_dt.jpg
019_.jpg
003_d.jpg
020_t.jpg
012_d.jpg
186_d.jpg
175_dt.jpg
184_t.jpg
022_.jpg
147_t.jpg
140_dt.jpg
126_.jpg
027_d.jpg
029_d.jpg
075_d.jpg
022_dt.jpg
130_.jpg
162_d.jpg
010_d.jpg
104_d.jpg
144_d.jpg
001_dt.jpg
004_t.jpg
040_dt.jpg
114_.jpg
151_d.jpg
094_d.jpg
072_dt.jpg
165_d.jpg
117_t.jpg
097_dt.jpg
169_dt.jpg
173_t.jpg
086_t.jpg
173_.jpg
072_d.jpg
120_dt.jpg
181_dt.jpg
031_.jpg
153_dt.jpg
080_t.jpg
128_t.jpg
070_d.jpg
094_t.jpg
123_.jpg
195_.jpg
063_.jpg
074_t.jpg
163_.jpg
056_t.jpg
060_d.jpg
122_dt.jpg
080_dt.jpg
042_dt.jpg
094_.jpg
071_dt.jpg
014_.jpg
141_.jpg
115_.jpg
178_dt.j

In [7]:
data.head()

,Img1,Img2,Target
0,099_.jpg,099_d.jpg,1
1,141_t.jpg,141_dt.jpg,1
2,186_t.jpg,186_dt.jpg,1
3,104_t.jpg,104_dt.jpg,1
4,193_t.jpg,058_t.jpg,0


这里下面是Base_Model_1 可以先不看，直接看Base_model_2

In [7]:
input_tensor = Input(shape=(224, 224, 3))
base_model = ResNet50(input_tensor=input_tensor,include_top=False,weights='imagenet')


/home/wenjie/anaconda3/envs/py36/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [8]:
for layer in base_model.layers:
    layer.trainable = True

In [8]:
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [9]:
model1 = Model(inputs=base_model.input, outputs=base_model.get_layer('activation_22').output)

这里下面是Base_Model_2 就是用来提取图片特征的

In [7]:
def Resblock_discriminator(layer_input,channels):
    h1 = Conv2D(channels,3,strides=1,padding='same')(layer_input)
    h1 = Activation('relu')(h1)
    h1 = Conv2D(channels,3,strides=1,padding='same')(h1)
    h1 = AveragePooling2D(pool_size=(2, 2))(h1)

    h2 = Conv2D(channels,1,strides=1,padding="valid")(layer_input)
    h2 = Activation('relu')(h2)
    h2 = AveragePooling2D(pool_size=(2, 2))(h2)
    return Add()([h2,h1])

def Resblock_discriminator_NP(layer_input,channels):
    h1 = Conv2D(channels,3,strides=1,padding='same')(layer_input)
    h1 = Activation('relu')(h1)
    h1 = Conv2D(channels,3,strides=1,padding='same')(h1)
    #h1 = AveragePooling2D(pool_size=(2, 2))(h1)

    h2 = Conv2D(channels,1,strides=1,padding="valid")(layer_input)
    h2 = Activation('relu')(h2)
    #h2 = AveragePooling2D(pool_size=(2, 2))(h2)
    return Add()([h2,h1])

In [8]:
def base_model(in_l):
    x = Conv2D(64,7,strides=2,padding='same')(in_l)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(3, 3),strides = 2)(x)
    
    x1 = Conv2D(64,3,padding='same')(x)
    x1 = BatchNormalization()(x1)
    x1 = Activation('relu')(x1)
    x2 = Conv2D(64,3,padding='same')(x1)
    x2 = BatchNormalization()(x2)
    x2 = Activation('relu')(x2)
    x = Add()([x,x2])
    x = Conv2D(128,3,padding='same')(x)
    """   
    x1 = Conv2D(128,3,padding='same')(x)
    x1 = BatchNormalization()(x1)
    x1 = Activation('relu')(x1)
    x2 = Conv2D(128,3,padding='same')(x1)
    x2 = BatchNormalization()(x2)
    x2 = Activation('relu')(x2)
    x = Add()([x,x2])
    x = MaxPooling2D(pool_size=(3, 3),strides = 2)(x)
    x = Conv2D(256,3,padding='same')(x)
    x1 = Conv2D(256,3,padding='same')(x)
    x1 = BatchNormalization()(x1)
    x1 = Activation('relu')(x1)
    x2 = Conv2D(256,3,padding='same')(x1)
    x2 = BatchNormalization()(x2)
    x2 = Activation('relu')(x2)
    x = Add()([x,x2])
    """
    x1 = Conv2D(128,3,padding='same')(x)
    x1 = BatchNormalization()(x1)
    x1 = Activation('relu')(x1)
    x2 = Conv2D(128,3,padding='same')(x1)
    x2 = BatchNormalization()(x2)
    x2 = Activation('relu')(x2)
    x = Add()([x,x2])
    return x
input_tensor = Input(shape=(224, 224, 3))
out_tensor = base_model(input_tensor)

In [9]:
model2 = Model(inputs = input_tensor,outputs = out_tensor)
model2.trainable = True

总的那个模型，在base_model的基础上接一个classifier跑的

In [10]:
def build_model(input_layer1,input_layer2,model):
    x1 = model(input_layer1)
    x2 = model(input_layer2)
    x3 = concatenate([x1,x2],axis = -1)  
    x3 = Resblock_discriminator(x3,512)
    x3 = Resblock_discriminator(x3,256)
    #x3 = Resblock_discriminator(x3,256)
    x3 = Conv2D(256,3,strides=2,padding="same")(x3)
    #x3 = Dense(256)(x3)
    #x3 = Dense(125)(x3)
    x3 = Dense(128)(x3)
    x3 = Flatten()(x3)
    output = Dense(2)(x3)
    output = Activation('softmax')(output)
    return output

input_layer1 = Input((224, 224, 3),name='input1' )
input_layer2 = Input((224, 224, 3),name='input2')

output_layer = build_model(input_layer1,input_layer2,model2)

In [11]:
model = Model(inputs = [input_layer1,input_layer2], outputs = output_layer)
model.summary()
model.trainable = True

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input1 (InputLayer)             (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input2 (InputLayer)             (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 55, 55, 128)  454144      input1[0][0]                     
                                                                 input2[0][0]                     
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 55, 55, 256)  0           model_1[1][0]                    
          

In [12]:
loss = Adam(0.0001,0.5)
model.compile(loss = 'categorical_crossentropy',optimizer = loss,metrics=["accuracy"])

In [14]:
data.shape

(3113, 3)

In [16]:
data.head()

,Img1,Img2,Target
0,116_t.jpg,116_d.jpg,1
1,071_.jpg,071_t.jpg,1
2,163_dt.jpg,151_t.jpg,0
3,142_t.jpg,142_dt.jpg,1
4,080_dt.jpg,080_.jpg,1


In [14]:
train_data = data.iloc[0:3000,:]

In [18]:
test_data = data.iloc[3000:-1,:]

In [16]:
pre = model.predict([x_1[0:10],x_2[0:10]])
print(pre)

[[0.49694276 0.50305724]
 [0.49522948 0.5047705 ]
 [0.49032295 0.50967705]
 [0.49492303 0.50507694]
 [0.49761844 0.5023815 ]
 [0.48109895 0.51890105]
 [0.47830197 0.52169806]
 [0.48127407 0.51872593]
 [0.4738773  0.52612275]
 [0.49068797 0.50931203]]


In [22]:
target[0:10]

array([[1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1]])

In [15]:
x_1 = np.array([img_dic[k] for k in train_data['Img1']])
x_2 = np.array([img_dic[k] for k in train_data['Img2']])
#target = np.array(train_data['Target'])
target = [[0,0] for i in range(train_data.shape[0])]
s = np.array(train_data['Target'])
for i in range(train_data.shape[0]):
    target[i][1-s[i]] = 1
target = np.array(target)

In [18]:
train_data.head()

,Img1,Img2,Target
0,099_.jpg,099_d.jpg,1
1,141_t.jpg,141_dt.jpg,1
2,186_t.jpg,186_dt.jpg,1
3,104_t.jpg,104_dt.jpg,1
4,193_t.jpg,058_t.jpg,0


In [17]:
w1 =  np.array(model.get_weights())

In [ ]:
model_checkpoint = ModelCheckpoint("./saved_model/direct.model", save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=1)
early_stopping = EarlyStopping(patience=20, verbose=1)
model.fit(x=[x_1,x_2],
            y=target,validation_split = 0.2,
            batch_size=16, epochs=100,verbose=1,
            callbacks=[early_stopping, model_checkpoint,reduce_lr]
         )

Train on 2400 samples, validate on 600 samples
Epoch 1/100
2400/2400 [==============================] - 39s 16ms/step - loss: 0.7699 - acc: 0.6350 - val_loss: 0.6549 - val_acc: 0.6750

Epoch 00001: val_loss improved from inf to 0.65489, saving model to ./saved_model/direct.model
Epoch 2/100
2400/2400 [==============================] - 36s 15ms/step - loss: 0.3745 - acc: 0.8442 - val_loss: 0.4048 - val_acc: 0.8117

Epoch 00002: val_loss improved from 0.65489 to 0.40479, saving model to ./saved_model/direct.model
Epoch 3/100
2400/2400 [==============================] - 36s 15ms/step - loss: 0.2289 - acc: 0.9062 - val_loss: 0.3867 - val_acc: 0.8333

Epoch 00003: val_loss improved from 0.40479 to 0.38673, saving model to ./saved_model/direct.model
Epoch 4/100
2400/2400 [==============================] - 36s 15ms/step - loss: 0.1654 - acc: 0.9337 - val_loss: 0.3231 - val_acc: 0.8733

Epoch 00004: val_loss improved from 0.38673 to 0.32313, saving model to ./saved_model/direct.model
Epoch 5/1

In [19]:
w2 = np.array(model.get_weights())

In [20]:
w = (w1 - w2)

In [21]:
w.shape

(50,)

In [28]:
w

array([array([[[[-1.91666931e-03, -7.74092972e-04, -4.40757349e-03, ...,
          -2.11894698e-03,  1.69716403e-03,  8.46233219e-04],
         [-9.08747315e-05, -1.18527003e-03, -1.51527673e-03, ...,
          -1.27210468e-03,  4.88353521e-03,  6.64368272e-05],
         [-8.63168389e-04, -4.72417101e-04,  1.40944496e-04, ...,
          -2.60281377e-03, -1.76206231e-06, -5.99907711e-04]],

        [[-2.14507058e-03, -1.30416080e-03, -4.37168125e-03, ...,
          -1.85427442e-03,  1.87576376e-03,  3.77096236e-04],
         [-6.33968040e-04, -1.49690779e-03, -1.30276196e-03, ...,
          -7.24717975e-04,  5.24847955e-03, -3.09487805e-04],
         [-1.16110500e-03, -6.43650070e-04,  3.32955271e-04, ...,
          -2.23185029e-03,  3.58335674e-04, -1.02658570e-03]],

        [[-2.98738480e-03, -1.08453818e-03, -4.15150356e-03, ...,
          -1.44559704e-03,  9.33799893e-04,  4.04376537e-04],
         [-1.88290700e-03, -1.13079511e-03, -8.23248178e-04, ...,
          -4.82294708e-05, 